In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from scipy.ndimage.filters import gaussian_filter

import vdmlab as vdm

In [ ]:
pos = dict()
pos['x'] = np.array([2, 4, 6, 8, 10, 8, 6])
pos['y'] = np.array([7, 5, 3, 1, 3, 5, 9])
pos['time'] = np.array([0., 1., 2., 3., 4., 5., 6.,])

plt.plot(pos['x'], pos['y'], '.', ms=10)
plt.xlim(0.5, 10.5)
plt.ylim(0.5, 10.5)
plt.show()

In [ ]:
binsize = 2
xedges = np.arange(pos['x'].min(), pos['x'].max(), binsize)
yedges = np.arange(pos['y'].min(), pos['y'].max(), binsize)
if (xedges[-1] != pos['x'].max()) or (yedges[-1] != pos['y'].max()):
    xedges = np.hstack((xedges, pos['x'].max()))
    yedges = np.hstack((yedges, pos['y'].max()))

In [ ]:
spikes = dict()
spikes['time'] = np.array([[0.1, 2.1, 2.3],
                           [0.2, 0.7, 1.1]])

In [ ]:
tuning_curves = tuning_curve_2d(spikes, pos, xedges, yedges, smooth=False)

In [ ]:
plt.figure()
xx, yy = np.meshgrid(xedges, yedges)
for tuning_curve in tuning_curves:
    pp = plt.pcolormesh(xx, yy, tuning_curve, cmap='YlGn')
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()

In [ ]:
a = np.array([[0,1,1],
              [3,4,5]])
a = a.flatten()
b = np.array([True, True, True, False, False, True])
c = np.array([0,0,0,0,0,0])

In [ ]:
c[b] = a[b] * a[b]
c

In [ ]:
def tuning_curve_2d(spikes, position, xedges, yedges, sampling_rate=1/30., gaussian_sigma=None):
    """Creates 2D tuning curves based on spikes and 2D position.
    
    Parameters
    ----------
    spikes : dict
        With time (floats) and labels (str) as keys. Where each inner array 
        represents the spike times for an individual neuron.
    position : dict
        With x (floats), y (floats), time (floats) as keys.
    xedges : np.array
    yedges : np.array
    sampling_rate : float
    gaussian_sigma : float
        Sigma used in gaussian filter if filtering.
    
    Returns
    -------
    tuning_curves : np.array
        Where each inner array is the tuning curve for an individual neuron.
    """
    position_2d, pos_xedges, pos_yedges = np.histogram2d(position['y'], position['x'], bins=[yedges, xedges])
    position_2d *= sampling_rate
    shape = position_2d.shape
#     position_2d = position_2d.flatten()
    occupied_idx = position_2d > 0

    count = 1
    tc = []
    for neuron_spikes in spikes['time']:
        print('Neuron:', count)
        count += 1
        spikes_x = []
        spikes_y = []
        for spike_time in neuron_spikes:
            spike_idx = vdm.find_nearest_idx(position['time'], spike_time)
            spikes_x.append(position['x'][spike_idx])
            spikes_y.append(position['y'][spike_idx])
        spikes_2d, spikes_xedges, spikes_yedges = np.histogram2d(spikes_y, spikes_x, bins=[yedges, xedges])
        
        firing_rate = np.zeros(shape)
#         firing_rate = firing_rate.flatten()
#         spikes_2d = spikes_2d.flatten()
        firing_rate[occupied_idx] = spikes_2d[occupied_idx] / position_2d[occupied_idx]
#         firing_rate = np.reshape(firing_rate, shape)

        tc.append(firing_rate)
        
    if gaussian_sigma is not None:
        tuning_curves = []
        for firing_rate in tc:
            tuning_curves.append(gaussian_filter(firing_rate, gaussian_sigma))
    else:
        print('Tuning curves with no filter.')
        tuning_curves = tc
    return tuning_curves

In [ ]:
from load_data import get_pos, get_spikes

import info.R063d3_info as r063d3

In [ ]:
info = r063d3

In [ ]:
pos = get_pos(info.pos_mat, info.pxl_to_cm)
spikes = get_spikes(info.spike_mat)

In [ ]:
binsize = 3
xedges = np.arange(pos['x'].min(), pos['x'].max()+binsize, binsize)
yedges = np.arange(pos['y'].min(), pos['y'].max()+binsize, binsize)
# if (xedges[-1] != pos['x'].max()) or (yedges[-1] != pos['y'].max()):
#     xedges = np.hstack((xedges, pos['x'].max()))
#     yedges = np.hstack((yedges, pos['y'].max()))

In [ ]:
speed = vdm.get_speed(pos)
t_run = speed['time'][speed['smoothed'] >= info.run_threshold]
run_idx = np.zeros(pos['time'].shape, dtype=bool)
for idx in t_run:
    run_idx |= (pos['time'] == idx)

run_pos = dict()
run_pos['x'] = pos['x'][run_idx]
run_pos['y'] = pos['y'][run_idx]
run_pos['time'] = pos['time'][run_idx]

In [ ]:
t_start = info.task_times['phase3'][0]
t_stop = info.task_times['phase3'][1]

t_start_idx = vdm.find_nearest_idx(run_pos['time'], t_start)
t_stop_idx = vdm.find_nearest_idx(run_pos['time'], t_stop)

sliced_pos = dict()
sliced_pos['x'] = run_pos['x'][t_start_idx:t_stop_idx]
sliced_pos['y'] = run_pos['y'][t_start_idx:t_stop_idx]
sliced_pos['time'] = run_pos['time'][t_start_idx:t_stop_idx]

sliced_spikes = dict()
sliced_spikes['time'] = vdm.time_slice(spikes['time'], t_start, t_stop)
sliced_spikes['label'] = spikes['label']

In [ ]:
plt.plot(sliced_pos['x'], sliced_pos['y'], 'g.', ms=1)
plt.show()

In [ ]:
phase_spikes = dict()
phase_spikes['time'] = sliced_spikes['time'][:5]

In [ ]:
tuning_curves = tuning_curve_2d(phase_spikes, sliced_pos, xedges, yedges, gaussian_sigma=0.5)

In [ ]:
import pickle
import os
pickle_filepath = 'C:\\Users\\Emily\\Code\\emi_shortcut\\cache\\pickled\\'
tc_filename = info.session_id + '_tuning_2d.pkl'
pickled_tc = os.path.join(pickle_filepath, tc_filename)

with open(pickled_tc, 'wb') as fileobj:
    pickle.dump(tuning_curves, fileobj)

In [ ]:
tuning_curves[0]

In [ ]:
def plot_2d(this, xedges, yedges):
    plt.figure()
    xx, yy = np.meshgrid(xedges, yedges)
    pp = plt.pcolormesh(xx, yy, this, cmap='YlGn')
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()

In [ ]:
# plt.figure()

xx, yy = np.meshgrid(xedges, yedges)
for tuning in tuning_curves[:9]:
#     plt.plot(sliced_pos['x'], sliced_pos['y'], 'b.', ms=1)
    pp = plt.pcolormesh(xx, yy, tuning, cmap='YlGn')
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()